<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_gptq_v2_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日英・英日翻訳サンプルコード(Japanese-English English-Japanese Translation sample code)


C3TR-Adapter Version2 gptq量子化版を使って日本語を英語、英語を翻訳するサンプルコードです。  
Sample code to translate Japanese to English and English to Japanese using C3TR-Adapter Version2 gptq quantized version.


上段メニューの「ランタイム」→「すべてのセルを実行」で実行してください  
Please execute it by clicking "Runtime" -> "Execute All Cells" in the upper menu  

もしくは各セルの[ ]の部分をクリックして実行してください  
Or click on the [ ] part of each cell to execute it.  


In [1]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [2]:
%%bash
git clone https://github.com/PanQiWei/AutoGPTQ.git && cd AutoGPTQ
pip install -vvv --no-build-isolation -e .

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-gnux17nu
Initialized build tracking at /tmp/pip-build-tracker-gnux17nu
Created build tracker: /tmp/pip-build-tracker-gnux17nu
Entered build tracker: /tmp/pip-build-tracker-gnux17nu
Created temporary directory: /tmp/pip-install-hfx9nkbc
Created temporary directory: /tmp/pip-ephem-wheel-cache-u6u5ydgs
Obtaining file:///content/AutoGPTQ
  Added file:///content/AutoGPTQ to build tracker '/tmp/pip-build-tracker-gnux17nu'
  Running setup.py (path:/content/AutoGPTQ/setup.py) egg_info for package from file:///content/AutoGPTQ
  Created temporary directory: /tmp/pip-pip-egg-info-d5z2id4z
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Source in /content/AutoGPTQ has version 0.8.0.dev0+cu1222, which satisfies requirement auto-gptq==0.8.0.dev0+cu1222 from file:///

Cloning into 'AutoGPTQ'...
  Running command python setup.py egg_info
  conda_cuda_include_dir /usr/lib/python3/dist-packages/nvidia/cuda_runtime/include
  running egg_info
  creating /tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info
  writing /tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-d5z2id4z/auto_gptq.egg-info/SOURCES.txt'
  /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:499: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
    warnings.warn(msg.format('we could not find ninja.'))
  reading manifest file '/tmp/pip-pip-egg-info

## インストール後、上段メニューより「ランタイム」→「セッションを再起動」してください
## After install, From the upper menu, click "runtime" -> "Restart Session".

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
model_name = "webbigdata/C3TR-Adapter_gptq"

# thanks to tk-master
# https://github.com/AutoGPTQ/AutoGPTQ/issues/406
config = AutoConfig.from_pretrained(model_name)
config.quantization_config["use_exllama"] = False
config.quantization_config["exllama_config"] = {"version":2}

# adjust your gpu memory size. 0 means first gpu.
max_memory={0: "15GiB", "cpu": "10GiB"}

model = AutoModelForCausalLM.from_pretrained(model_name
        , torch_dtype=torch.float16  # change torch.bfloat16 if you use Paid colab or GPU support bfloat16.
        , device_map="auto", max_memory=max_memory
        , config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.unk_token

prompt_text = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:
Translate Japanese to English.
When translating, please use the following hints:
[writeing_style: web-fiction]
[リュグナー: Lügner]
[フリーレン: Frieren]
[フリーレン_character_style: female, casual]
[葬送のフリーレン: Frieren at the Funeral]
[フェルン: Fern]
[ゾルトラーク: Soul track]

### Input:
リュグナー「あの小娘の所作には面影があった。私は昔、同じ魔法を受けたことがある。・・・そうか、思い出した。フリーレンだ。人類のゾルトラークの研究解析に大きく貢献し、歴史上で最も多くの魔族を葬り去った魔法使い。葬送のフリーレン。私の嫌いな天才だ。」

・・・

フリーレン：魔法は好き？
フェルン：ほどほどでございます。
フリーレン：私と同じだ。

...

フェルン：フリーレン様は本当に魔法がお好きなのですね。
フリーレン：ほどほどだよ。フェルンと同じで。
フェルン：少し違うような気がします。
フリーレン：同じだよ。
### Response:
"""

tokens = tokenizer(prompt_text, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).to("cuda:0").input_ids

output = model.generate(
        input_ids=tokens,
        max_new_tokens=800,
        do_sample=True,
        num_beams=3, temperature=0.5, top_p=0.3,
        repetition_penalty=1.0)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when poss

model.safetensors.index.json:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/616M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

<bos>You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

In [3]:

prompt_text = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:
Translate English to Japanese.
When translating, please use the following hints:
[writeing_style: casual]
[リュグナー: Lügner]
[フリーレン: Frieren]
[Frieren at the Funeral: 葬送のフリーレン]
[フリーレン_character_style: female, casual]
[フェルン: Fern]
[フェルン_character_style: female, humility]
[フェルン_first_person_and_ending: 私, ございます]
[Soul track: ゾルトラーク]

### Input:
Lügner: There was something familiar in that young girl's demeanor. I once received the same magic... Ah, I remember now. Frieren. The mage who contributed greatly to the research and analysis of the human Soul track and laid to rest the most demons in history. Frieren at the Funeral. My least favorite genius.

...

Frieren: Do you like magic?
Fern: Somewhat.
Frieren: The same as me.

...

Fern: Lady Frieren really likes magic, doesn't she?
Frieren: Somewhat. The same as Fern.
Fern: It seems a little different.
Frieren: The same

### Response:
"""

tokens = tokenizer(prompt_text, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).to("cuda:0").input_ids

output = model.generate(
        input_ids=tokens,
        max_new_tokens=800,
        do_sample=True,
        num_beams=3, temperature=0.5, top_p=0.3,
        repetition_penalty=1.0)
print(tokenizer.decode(output[0]))

<bos>You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th